# Limpieza y Análisis de Datos con Pandas

Este notebook trabaja con el archivo `screentime_analysis_extended.csv`.

Objetivos:
1. Cargar datos y explorarlos.
2. Detectar y tratar valores nulos.
3. Eliminar duplicados.
4. Arreglar tipos de datos.
5. Limpiar texto.
6. Detectar columnas vacías y borrarlas.
7. Guardar el dataset limpio.
8. Hacer un mini análisis final.

## 1. Importación y exploración inicial

In [ ]:
import pandas as pd

# Cargar el CSV (ajusta la ruta si lo tienes en otra carpeta)
df = pd.read_csv("screentime_analysis_extended.csv")

# Vista rápida de las primeras filas
display(df.head())

# Información general de columnas y tipos
df.info()

# Estadísticas básicas (incluyendo texto)
display(df.describe(include='all'))


## 2. Detección y manejo de valores nulos (NaN)

In [ ]:
# Conteo de valores nulos por columna
print("Valores nulos por columna:")
print(df.isna().sum())

# Rellenar NaN en columnas numéricas clave con valores por defecto
df.fillna({'screen_time(min)': 0, 'notifications': 0}, inplace=True)

# Eliminar filas que no tengan app o tiempo de pantalla (esenciales para análisis)
df.dropna(subset=['app_name', 'screen_time(min)'], inplace=True)

print('\nDespués de limpieza de NaN:')
print(df.isna().sum())

display(df.head())


## 3. Eliminación de duplicados

In [ ]:
# Detectar filas duplicadas completas
duplicados = df[df.duplicated()]
print("Filas duplicadas detectadas:")
display(duplicados)

# Eliminar duplicados
df.drop_duplicates(inplace=True)

print("Tamaño tras eliminar duplicados:", df.shape)


## 4. Conversión de tipos de datos (`astype`, `to_datetime`)

In [ ]:
print("Tipos ANTES:")
print(df.dtypes)

# Convertir columnas numéricas que puedan venir como texto
df['screen_time(min)'] = df['screen_time(min)'].astype(float)

# notifications puede tener mezcla de enteros y strings -> forzamos conversión segura
df['notifications'] = pd.to_numeric(df['notifications'], errors='coerce').fillna(0).astype(int)

# Convertir fechas a datetime, forzando formato cuando hay mezcla
df['date'] = pd.to_datetime(df['date'], errors='coerce', dayfirst=True)

print("\nTipos DESPUÉS:")
print(df.dtypes)

display(df.head())


## 5. Eliminación de columnas vacías

In [ ]:
# Eliminamos columnas que estén completamente vacías
df.dropna(axis=1, how='all', inplace=True)

print("Columnas actuales del DataFrame:")
print(df.columns.tolist())


## 6. Limpieza y normalización de texto (`.str` en Pandas)

In [ ]:
# Normalizar nombres de apps:
# - quitar espacios en blanco
# - poner la primera letra en mayúscula de cada palabra
# - eliminar espacios internos (Instagram  -> Instagram, 'YouTube Music' -> 'Youtubemusic')
df['app_name'] = (
    df['app_name']
      .astype(str)
      .str.strip()
      .str.title()
      .str.replace(' ', '', regex=False)
)

display(df[['app_name']].head(10))

# Buscar todas las filas que contienen "Youtube"
youtube_rows = df[df['app_name'].str.contains('Youtube', case=False, na=False)]
print("Filas con Youtube:")
display(youtube_rows)


## 7. Métodos extra de cadena (`str.len`, `str.startswith`, etc.)

In [ ]:
# Longitud del nombre de la app
df['app_name_len'] = df['app_name'].str.len()

# Apps que empiezan por 'Y'
apps_y = df[df['app_name'].str.startswith('Y', na=False)]

print("Apps que empiezan por 'Y':")
display(apps_y[['app_name', 'app_name_len']].drop_duplicates())

display(df[['app_name', 'app_name_len']].head())


## 8. Detección de tipos mezclados en columnas

In [ ]:
print("Tipos actuales:")
print(df.dtypes)

print("\nTipo real de cada celda en las primeras filas:")
display(df.applymap(type).head())

# Convertir automáticamente al tipo más adecuado (opcional)
df = df.convert_dtypes()

print("\nTipos tras convert_dtypes():")
print(df.dtypes)


## 9. Guardar el dataset limpio

In [ ]:
df.to_csv('screentime_analysis_clean.csv', index=False)
print('Archivo screentime_analysis_clean.csv guardado correctamente.')

## 10. Mini análisis final por aplicación

In [ ]:
# ¿Qué apps tienen más tiempo medio de pantalla?
screen_time_media = (
    df.groupby('app_name')['screen_time(min)']
      .mean()
      .sort_values(ascending=False)
)

print("Tiempo medio de pantalla (min) por app:")
display(screen_time_media)

# ¿Qué apps generan más notificaciones?
notif_media = (
    df.groupby('app_name')['notifications']
      .mean()
      .sort_values(ascending=False)
)

print("Notificaciones medias por app:")
display(notif_media)
